# Seismic Data Analysis with PyTorch

This notebook demonstrates how to use the gaia-landslides-detect package for seismic data analysis and model deployment.

## Setup

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import torch
from pathlib import Path

# Add src to path
sys.path.insert(0, '../src')

from models import SeismicCNN, load_model, save_model
from data import SeismicDataProcessor, SeismicDataset
from utils import plot_seismogram, plot_spectrogram, get_device, set_seed

# Set random seed for reproducibility
set_seed(42)

# Get device
device = get_device()

print("Setup complete!")

## 1. Create a Synthetic Seismogram

For demonstration purposes, we'll create a synthetic 3-component seismogram.

In [ ]:
# Create synthetic seismic data
sampling_rate = 100.0  # Hz
duration = 30.0  # seconds
n_samples = int(sampling_rate * duration)
time = np.arange(n_samples) / sampling_rate

# Create 3-component synthetic waveform with some noise and a signal
np.random.seed(42)
n_channels = 3
data = np.zeros((n_channels, n_samples))

for i in range(n_channels):
    # Background noise
    noise = np.random.randn(n_samples) * 0.1
    
    # Add a synthetic signal (simulating a seismic event)
    t_event = 10.0  # Event starts at 10 seconds
    freq = 5.0 + i  # Different frequencies for different components
    signal_window = (time >= t_event) & (time <= t_event + 5.0)
    envelope = np.exp(-0.5 * (time[signal_window] - t_event - 2.5) ** 2)
    signal = np.zeros(n_samples)
    signal[signal_window] = envelope * np.sin(2 * np.pi * freq * (time[signal_window] - t_event))
    
    data[i] = noise + signal

print(f"Created synthetic seismogram with shape: {data.shape}")
print(f"  - {n_channels} components")
print(f"  - {n_samples} samples ({duration} seconds at {sampling_rate} Hz)")

## 2. Visualize the Seismogram

In [ ]:
# Plot the seismogram
fig = plot_seismogram(
    data,
    sampling_rate=sampling_rate,
    title="Synthetic 3-Component Seismogram",
    labels=['Vertical (Z)', 'North (N)', 'East (E)'],
    save_path='../plots/example_seismogram.png'
)
plt.show()

## 3. Create and Visualize Spectrogram

In [ ]:
# Plot spectrogram
fig = plot_spectrogram(
    data,
    sampling_rate=sampling_rate,
    title="Spectrogram of Synthetic Seismogram",
    save_path='../plots/example_spectrogram.png'
)
plt.show()

## 4. Initialize the Model

Create a SeismicCNN model for classification.

In [ ]:
# Create model
model = SeismicCNN(
    input_channels=3,
    num_classes=2,  # Binary classification (e.g., landslide vs. no landslide)
    dropout=0.5
)

model = model.to(device)

# Print model architecture
print("Model Architecture:")
print(model)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nTotal parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

## 5. Make Predictions

Demonstrate inference with the model.

In [ ]:
# Prepare data for model input
data_tensor = torch.from_numpy(data).float().unsqueeze(0).to(device)  # Add batch dimension
print(f"Input tensor shape: {data_tensor.shape}")

# Make prediction
model.eval()
with torch.no_grad():
    output = model(data_tensor)
    probabilities = torch.softmax(output, dim=1)

print(f"\nModel output (logits): {output}")
print(f"Probabilities: {probabilities}")
print(f"Predicted class: {torch.argmax(probabilities, dim=1).item()}")

## 6. Data Processing with ObsPy

Demonstrate the data processing pipeline.

In [ ]:
# Initialize data processor
processor = SeismicDataProcessor(
    sampling_rate=100.0,
    window_length=30.0,
    normalize=True
)

# Process the data
processed_data = processor._normalize(data, method='std')
print(f"Processed data shape: {processed_data.shape}")
print(f"Mean: {np.mean(processed_data, axis=1)}")
print(f"Std: {np.std(processed_data, axis=1)}")

# Convert to PyTorch tensor
tensor_data = processor.to_torch(processed_data)
print(f"\nTensor shape: {tensor_data.shape}")
print(f"Tensor dtype: {tensor_data.dtype}")

## 7. Save and Load Models

Demonstrate saving and loading model checkpoints.

In [ ]:
# Save the model
save_path = '../checkpoints/example_model.pth'
Path('../checkpoints').mkdir(exist_ok=True)

save_model(
    model,
    save_path,
    epoch=0,
    loss=0.5,
    metadata={'description': 'Example seismic CNN model', 'version': '0.1.0'}
)
print(f"Model saved to {save_path}")

# Load the model
loaded_model = load_model(
    save_path,
    model_class=SeismicCNN,
    device=str(device),
    input_channels=3,
    num_classes=2
)
print("Model loaded successfully!")

# Verify loaded model works
with torch.no_grad():
    output_loaded = loaded_model(data_tensor)
    
print(f"\nOriginal output: {output}")
print(f"Loaded model output: {output_loaded}")
print(f"Outputs match: {torch.allclose(output, output_loaded)}")

## Summary

This notebook demonstrated:
1. Creating synthetic seismic data
2. Visualizing seismograms and spectrograms
3. Creating and using a PyTorch model for seismic data classification
4. Processing seismic data with the data pipeline
5. Saving and loading model checkpoints

## Next Steps

- Load real seismic data using ObsPy or SeisBench
- Train the model on your dataset
- Experiment with different model architectures
- Implement custom data augmentation techniques
- Deploy the model for real-time inference